In [18]:
import pandas as pd
import numpy as np
import gc
import time
from pandas.core.common import SettingWithCopyWarning
import warnings
from pandas.io.json import json_normalize
import json
import os
warnings.simplefilter('error', SettingWithCopyWarning)
gc.enable()
%matplotlib inline

In [25]:
features = ['channelGrouping', 'date', 'fullVisitorId', 'visitId',\
       'visitNumber', 'visitStartTime', 'device_browser',\
       'device_deviceCategory', 'device_isMobile', 'device_operatingSystem',\
       'geoNetwork_city', 'geoNetwork_continent', 'geoNetwork_country',\
       'geoNetwork_metro', 'geoNetwork_networkDomain', 'geoNetwork_region',\
       'geoNetwork_subContinent', 'totals_bounces', 'totals_hits',\
       'totals_newVisits', 'totals_pageviews', 'totals_transactionRevenue',\
       'trafficSource_adContent', 'trafficSource_campaign',\
       'trafficSource_isTrueDirect', 'trafficSource_keyword',\
       'trafficSource_medium', 'trafficSource_referralPath',\
       'trafficSource_source','trafficSource_adwordsClickInfo.page',\
       'trafficSource_adwordsClickInfo.adNetworkType',
       'trafficSource_adwordsClickInfo.isVideoAd',
       'trafficSource_adwordsClickInfo.slot',
           ]

In [26]:
gc.enable()
def load_df(csv_path='/home/baitong/pywork/RevenuePrediction/all (1)/train_v2.csv'):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    ans = pd.DataFrame()
    dfs = pd.read_csv(csv_path, sep=',',
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                    chunksize = 100000)
    for df in dfs:
        df.reset_index(drop = True,inplace = True)
        for column in JSON_COLUMNS:
            column_as_df = json_normalize(df[column])
            column_as_df.columns = [f"{column}_{subcolumn}" for subcolumn in column_as_df.columns]
            df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)

        print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
        use_df = df[features]
        del df
        gc.collect()
        ans = pd.concat([ans, use_df], axis = 0).reset_index(drop = True)
        print(ans.shape)
    return ans

train = load_df()
train.shape

Loaded train_v2.csv. Shape: (100000, 59)
(100000, 33)
Loaded train_v2.csv. Shape: (100000, 60)
(200000, 33)
Loaded train_v2.csv. Shape: (100000, 59)
(300000, 33)
Loaded train_v2.csv. Shape: (100000, 59)
(400000, 33)
Loaded train_v2.csv. Shape: (100000, 59)
(500000, 33)
Loaded train_v2.csv. Shape: (100000, 59)
(600000, 33)
Loaded train_v2.csv. Shape: (100000, 59)
(700000, 33)
Loaded train_v2.csv. Shape: (100000, 59)
(800000, 33)
Loaded train_v2.csv. Shape: (100000, 59)
(900000, 33)
Loaded train_v2.csv. Shape: (100000, 59)
(1000000, 33)
Loaded train_v2.csv. Shape: (100000, 59)
(1100000, 33)
Loaded train_v2.csv. Shape: (100000, 59)
(1200000, 33)
Loaded train_v2.csv. Shape: (100000, 59)
(1300000, 33)
Loaded train_v2.csv. Shape: (100000, 59)
(1400000, 33)
Loaded train_v2.csv. Shape: (100000, 59)
(1500000, 33)
Loaded train_v2.csv. Shape: (100000, 59)
(1600000, 33)
Loaded train_v2.csv. Shape: (100000, 59)
(1700000, 33)
Loaded train_v2.csv. Shape: (8337, 59)
(1708337, 33)


(1708337, 33)

In [27]:
path = '/home/baitong/pywork/RevenuePrediction/all (1)'

In [28]:
test = load_df("/home/baitong/pywork/RevenuePrediction/all (1)/test_v2.csv")

Loaded test_v2.csv. Shape: (100000, 59)
(100000, 33)
Loaded test_v2.csv. Shape: (100000, 59)
(200000, 33)
Loaded test_v2.csv. Shape: (100000, 59)
(300000, 33)
Loaded test_v2.csv. Shape: (100000, 59)
(400000, 33)
Loaded test_v2.csv. Shape: (1589, 59)
(401589, 33)


In [ ]:
train.to_csv(path +"/train.csv", index=False)
test.to_csv(path+"/test.csv", index = False)
train.head()

In [11]:
test.shape,train.shape

((401589, 29), (1708337, 29))